In [ ]:
# devtools::install_github("cellgeni/sceasy")

In [8]:
library(scater)
library(scDblFinder)
library(BiocParallel)
source("../tools/formating/formating.R")

In [15]:
# path = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat"
path = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad"
# srat <- LoadSeurat(path)
# srat

In [16]:
srat <- tryCatch(
        LoadSeurat(path),
        error = function(e) {
            stop("The file format is not supported.")
            print(e)
        }
    )

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
X -> counts; scale.data -> data



In [17]:
srat

An object of class Seurat 
23341 features across 2392 samples within 1 assay 
Active assay: RNA (23341 features, 0 variable features)
 2 layers present: counts, data
 3 dimensional reductions calculated: pca, tsne, umap

In [ ]:
UMAPPlot(object = srat, label=T)

In [3]:
srat@meta.data

,orig.ident,nCount_RNA,nFeature_RNA,nCounts_RNA,nFeatures_RNA,channel,tissue,subtissue,mouse.sex,mouse.id,⋯,cluster.ids,cell_ontology_id,percent.mt,percent.rb,percent.hb,percent.plat,RNA_snn_res.0.5,seurat_clusters,doublet_score,doublet_class
,<fct>,<dbl>,<int>,<dbl>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>
10X_P4_3_AAAGTAGAGATGCCAG,10X,10291,2853,10291,2853,10X_P4_3,Bladder,,M,3-M-8,⋯,0,CL:1001319,0,0,0,0,3,3,0.2572535276,singlet
10X_P4_3_AACTCCCGTCGGGTCT,10X,12944,3347,12944,3347,10X_P4_3,Bladder,,M,3-M-8,⋯,5,CL:1001319,0,0,0,0,5,5,0.1914071590,singlet
10X_P4_3_AACTCTTAGTTGCAGG,10X,19594,3851,19594,3851,10X_P4_3,Bladder,,M,3-M-8,⋯,3,CL:1001428,0,0,0,0,4,4,0.4951692522,singlet
10X_P4_3_AACTCTTTCATAACCG,10X,14594,3809,14594,3809,10X_P4_3,Bladder,,M,3-M-8,⋯,2,CL:1001319,0,0,0,0,0,0,0.2093610018,singlet
10X_P4_3_AAGACCTAGATCCGAG,10X,19190,3608,19190,3608,10X_P4_3,Bladder,,M,3-M-8,⋯,6,CL:0000738,0,0,0,0,8,8,0.3869006932,singlet
10X_P4_3_AAGGAGCGTGCAACTT,10X,5005,1960,5005,1960,10X_P4_3,Bladder,,M,3-M-8,⋯,0,CL:1001319,0,0,0,0,3,3,0.0268953685,singlet
10X_P4_3_AAGTCTGAGATAGTCA,10X,6253,2236,6253,2236,10X_P4_3,Bladder,,M,3-M-8,⋯,4,CL:1001428,0,0,0,0,4,4,0.0088629527,singlet
10X_P4_3_ACAGCTATCATACGGT,10X,15910,3374,15910,3374,10X_P4_3,Bladder,,M,3-M-8,⋯,1,CL:1001428,0,0,0,0,2,2,0.1332673728,singlet
10X_P4_3_ACATACGCAGCTCCGA,10X,2881,1558,2881,1558,10X_P4_3,Bladder,,M,3-M-8,⋯,4,CL:1001428,0,0,0,0,2,2,0.0018370392,singlet


In [ ]:
colnames(srat[['RNA']])

In [ ]:
srat[['pca']]

In [ ]:
srat <- Seurat::ReadH5AD(path)
srat

In [ ]:
adata <- LoadAnndata(path)
adata

In [ ]:
# Add dimension reduction
  embed_names <- unlist(adata$obsm_keys())
  if (length(embed_names) > 0) {
    embeddings <- sapply(embed_names, function(x) as.matrix(adata$obsm[[x]]), simplify = FALSE, USE.NAMES = TRUE)
    names(embeddings) <- embed_names
      for (name in embed_names) {
        rownames(embeddings[[name]]) <- colnames(srat[['RNA']])
      }
  }

In [ ]:
embeddings

In [ ]:
length(adata$obsm[['X_tsne']])

In [ ]:
adata$obsm[['X_tsne']]

In [ ]:
adata$layers

In [ ]:
adata$layers <- NULL
adata

In [ ]:
'' %in% names(adata$layers)

In [ ]:
names(adata$layers)

In [ ]:
for (layer in names(adata$layers)){
    print(layer)
  }

In [ ]:
is.null(names(adata$layers))

In [ ]:
names(adata$layers) == 'aaa'

In [ ]:
'scale.data' %in% names(adata$layers)

In [ ]:
adata$layers['scale.data']

In [ ]:
srat <- LoadSeurat("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat")
srat

In [ ]:
print(srat[['umap']])

In [ ]:
dims=1:10
srat <- FindVariableFeatures(srat, selection.method = "vst")
srat <- ScaleData(srat, features = rownames(srat))

# PCA
# srat <- RunPCA(srat, features = VariableFeatures(srat), ndims.print = 6:10, nfeatures.print = 10)
srat <- RunPCA(srat, features = VariableFeatures(srat))

srat <- FindNeighbors(srat, dims=dims)
srat <- FindClusters(srat, resolution = 0.5)
srat

# TSNE
srat <- RunTSNE(srat, dims=dims)
# UMAP
srat <- RunUMAP(srat, dims=dims)

umap <- Embeddings(object = srat, reduction = "umap")

In [ ]:
embed_names <- unlist(adata$obsm_keys())
embed_names

In [ ]:
adata$obsm[['X_umap']]

In [ ]:
adata$obsm[['X_pca']]

In [ ]:
as.matrix(adata$obsm[['X_pca']])

In [ ]:
adata_Seurat <- CreateSeuratObject(counts = t(adata$X), meta.data = adata$obs)
adata_Seurat

In [ ]:
adata_Seurat$data

In [ ]:
if (reticulate::py_to_r(sp$issparse(adata$X))) {
      X <- Matrix::t(reticulate::py_to_r(sp$csc_matrix(adata$X)))
    } else {
      X <- t(reticulate::py_to_r(adata$X))
    }

In [ ]:
adata$X

In [ ]:
.obs2metadata <- function(obs_pd, assay = "RNA") {
  obs_df <- .regularise_df(obs_pd, drop_single_values=FALSE, drop_na_values=TRUE)
  colnames(obs_df) <- sub("n_counts", paste0("nCounts_", assay), colnames(obs_df))
  colnames(obs_df) <- sub("n_genes", paste0("nFeaturess_", assay), colnames(obs_df))
  return(obs_df)
}

#' Prepare feature metadata
#'
#' This function prepare feature metadata from AnnData.var
#'
#' @param var_pd Input AnnData.var dataframe
#'
#' @return AnnData object
#'
#' @import reticulate
.var2feature_metadata <- function(var_pd) {
  var_df <- .regularise_df(var_pd, drop_single_values=FALSE, drop_na_values=TRUE)
  colnames(var_df) <- sub("dispersions_norm", "mvp.dispersion.scaled", colnames(var_df))
  colnames(var_df) <- sub("dispersions", "mvp.dispersion", colnames(var_df))
  colnames(var_df) <- sub("means", "mvp.mean", colnames(var_df))
  colnames(var_df) <- sub("highly_variable", "highly.variable", colnames(var_df))
  return(var_df)
}

In [ ]:
obs_df <- .obs2metadata(adata$obs)
var_df <- .var2feature_metadata(adata$var)
X <- t(adata$X)
colnames(X) <- rownames(obs_df)
rownames(X) <- rownames(var_df)
X

In [ ]:
rownames(obs_df)

In [ ]:
rownames(var_df)

In [ ]:
obs_df

In [ ]:
x_t <- Matrix::t(adata$X)
x_t

In [ ]:
adata_Seurat$RNA$counts

In [ ]:
adata_Seurat$RNA$counts

In [ ]:
Convert(path, "h5seurat", overwrite = TRUE)

In [ ]:
srat <- LoadH5Seurat(paste0(tools::file_path_sans_ext(path), ".h5seurat"), meta.data = FALSE, misc = FALSE)

In [ ]:
library(Seurat)
library(SingleCellExperiment)
library(SeuratDisk)
library(SeuratData)
input = "/ps/ai-ready/data/error/tung.rds"
robj <- readRDS(input)
robj

In [ ]:
logcounts(robj)

In [ ]:
'logcounts'  %in% names(robj)

In [ ]:
srat <- as.Seurat(robj, slot = "counts")

In [ ]:
# path <- "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj"
path <- "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat"

In [ ]:
srat <- LoadSeurat(path)
srat

In [ ]:
adata <- SeuratToAnndata(srat)
adata

In [ ]:
adata$X

In [ ]:
obj <- srat
default_assay <- 'RNA'
if(paste0("nCount_", default_assay) %in% names(obj@meta.data)) names(obj@meta.data)[names(obj@meta.data) ==paste0("nCount_", default_assay)] <-"n_counts"
if(paste0("nFeature_", default_assay) %in% names(obj@meta.data)) names(obj@meta.data)[names(obj@meta.data) ==paste0("nFeature_", default_assay)] <-"n_genes"
if("percent.mt" %in% names(obj@meta.data)) names(obj@meta.data)[names(obj@meta.data) =="percent.mt"] <-"percent_mito"
obj@meta.data

In [ ]:
adata$layers['scale.data']

In [ ]:
srat[['RNA']]@counts

In [ ]:
mat <- Seurat::GetAssayData(object=srat, assay='RNA', layer='X')

In [ ]:
default_assay <- DefaultAssay(srat)
default_assay

In [ ]:
adata <- LoadAnndata("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad")
adata

In [ ]:
adata$obsm['X_pca']

In [ ]:
Embeddings(object = srat, reduction = "pca")

In [ ]:
max(srat[['RNA']]@counts)

In [ ]:
min(srat[['RNA']]@counts)

In [ ]:
doublet_annnotation <- AnnotateDroplet(srat[[default_assay]]@counts)

In [ ]:
doublet_annnotation$doublet_score

In [ ]:
srat[["Doublet_score"]] <- doublet_annnotation$doublet_score
srat[["Doublet_class"]] <- doublet_annnotation$doublet_class

In [ ]:
data_mat = srat[['RNA']]@counts

In [ ]:
# -i data_mat -o doublet_score -o doublet_class

set.seed(123)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_mat),
    ) 
)
doublet_score = sce$scDblFinder.score
doublet_class = sce$scDblFinder.class

In [ ]:
srat[["Doublet_score"]] <- doublet_score

In [ ]:
srat[["Doublet_class"]] <- doublet_class

In [ ]:
srat[[]]

In [ ]:
dim(srat[['RNA']]@counts)
dim(srat)
dim(srat@meta.data)

In [ ]:
if(!paste0("nCount_", default_assay) %in% names(x = srat[[]])) srat[[paste0("nCount_", default_assay)]] <- colSums(x = srat[[default_assay]], slot = "counts")  # nCount of the default assay
if(!paste0("nFeature_", default_assay) %in% names(x = srat[[]])) srat[[paste0("nFeature_", default_assay)]] <- colSums(x = GetAssayData(object = srat[[default_assay]], slot = "counts") > 0)  # nFeature of the default assay

In [ ]:
head(srat@meta.data)

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat, filename = output_path, overwrite = TRUE, verbose = TRUE)

In [ ]:
srat1 <- LoadH5Seurat(output_path)
srat1

In [ ]:
srat1@meta.data

In [ ]:
adata_path <- Convert('/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat', dest = "h5ad" , overwrite = TRUE)
adata_path

In [ ]:
# Calculate the percentage of mitocondrial per cell and add to the metadata.
if(! "percent.mt" %in% names(x = srat[[]])) srat[["percent.mt"]] <- PercentageFeatureSet(srat, pattern = "^MT-")
# Calculate the proportion gene expression that comes from ribosomal proteins.
if(! "percent.rb" %in% names(x = srat[[]])) srat[["percent.rb"]] <- PercentageFeatureSet(srat, pattern = "^RP[SL]")

# Percentage hemoglobin genes - includes all genes starting with HB except HBP.
if(! "percent.hb" %in% names(x = srat[[]])) srat[["percent.hb"]] <- PercentageFeatureSet(srat, pattern = "^HB[^(P)]")
if(! "percent.plat" %in% names(x = srat[[]])) srat[["percent.plat"]] <- PercentageFeatureSet(srat, pattern = "PECAM1|PF4")
head(srat@meta.data)

In [ ]:
# Add the doublet annotation
doublets <- read.table(here::here('api/tools/qc/scrublet_calls.tsv'), header = F, row.names = 1)
colnames(doublets) <- c("Doublet_score", "Is_doublet")
srat <- AddMetaData(srat, doublets)
# srat[['Is_doublet']] <- !is.na(srat[['Is_doublet']])

In [ ]:
head(srat@meta.data)

In [ ]:
srat@meta.data

In [ ]:
names(srat@meta.data)[names(srat@meta.data) =="nCount_RNA"] <-"n_counts"
srat@meta.data

In [ ]:
names(srat@meta.data)

In [ ]:
nFeature_min=200
nFeature_max=0
percent_mt_max=5
percent_rb_min=0
srat <- subset(srat, subset = paste0("nFeature_", default_assay) > nFeature_min & percent.mt < percent_mt_max)
if(nFeature_max != 0) srat <- subset(srat, subset = paste0("nFeature_", default_assay) < nFeature_max)
if(percent_rb_min != 0)  srat <- subset(srat, subset = percent.rb > percent_rb_min)

In [ ]:
srat <- subset(srat, subset = Is_doublet != 'True' | is.na(Is_doublet))

In [ ]:
#' Regularise dataframe
#'
#' This function checks if certain columns of a dataframe is of a single value
#' and drop them if required
#'
#' @param df Input data frame, usually cell metadata table (data.frame-like
#'   object)
#' @param drop_single_values Drop columns with only a single value (logical)
#'
#' @return Dataframe
.regularise_df <- function(df, drop_single_values=FALSE, drop_na_values=TRUE) {
  if (ncol(df) == 0) df[["name"]] <- rownames(df)
  if (drop_single_values) {
    k_singular <- sapply(df, function(x) length(unique(x)) == 1)
    if (sum(k_singular) > 0) {
      warning(
        paste("Dropping single category variables:"),
        paste(colnames(df)[k_singular], collapse = ", ")
      )
    }
    df <- df[, !k_singular, drop = F]
    if (ncol(df) == 0) df[["name"]] <- rownames(df)
  }
 if (drop_na_values) {
    k_na <- sapply(srat@meta.data, function(x) sum(is.na(x))==length(x))
    if (sum(k_na) > 0) {
      warning(
        paste("Dropping NA category variables:"),
        paste(colnames(df)[k_na], collapse = ", ")
      )
    }
    df <- df[, !k_na, drop = F]
    if (ncol(df) == 0) df[["name"]] <- rownames(df)
  }
  return(df)
}

In [ ]:
output_path_rds <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds'
saveRDS(srat, output_path_rds)
srat1 <- LoadSeurat(output_path_rds)
srat1

In [ ]:
k_na <- sapply(srat@meta.data, function(x) sum(is.na(x))==length(x))
k_na
sum(k_na)

In [ ]:
srat_qc@meta.data %>% is.na()

In [ ]:
k_singular <- sapply(srat_qc@meta.data, function(x) length(unique(x)) == 1)
k_singular

In [ ]:
srat_qc@meta.data

In [ ]:
srat@meta.data <- .regularise_df(srat@meta.data, drop_single_values=FALSE, drop_na_values=TRUE)
srat@meta.data

In [ ]:
srat_qc@meta.data <- .regularise_df(srat_qc@meta.data, drop_single_values = FALSE)
srat_qc@meta.data

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat_qc, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat_3 <- LoadH5Seurat(output_path)
srat_3

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat1, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat_2 <- LoadH5Seurat(output_path)
srat_2

In [ ]:
srat1[["meta.data"]]

In [ ]:
srat1@meta.data

In [ ]:
devtools::install_github("cellgeni/sceasy")

In [ ]:
sceasy::convertFormat(srat1, from="seurat", to="anndata",
                       outFile='/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad')

In [ ]:
adata <- LoadAnndata('/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad')
adata

In [ ]:
adata$obs

In [ ]:
adata$X

In [ ]:
srat_qc <- CreateSeuratObject(counts=srat[['RNA']]@counts, meta.data=srat@meta.data, project = Project(srat))
srat_qc

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat_qc, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat_qc_1 <- LoadH5Seurat(output_path)
srat_qc_1

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat1 <- LoadH5Seurat(output_path)
srat1

In [ ]:
srat <- subset(srat, subset = Is_doublet != 'True')

In [ ]:
srat[["meta.data"]]

In [ ]:
Cells(srat)

In [ ]:
srat <- NormalizeData(srat, normalization.method = "LogNormalize", scale.factor = 10000)
srat <- FindVariableFeatures(srat, selection.method = "vst")
srat <- ScaleData(srat, features = rownames(srat))
srat
dim(srat)
dim(srat@meta.data)

In [ ]:
srat[['Doublet_score']][sapply(srat[['Doublet_score']], is.na)] <- 0
srat[['Doublet_score']]

In [ ]:
srat[['Doublet_score']] <- !is.na(srat[['Doublet_score']])


In [ ]:
# srat <- subset(srat, subset = percent.rb >= 0)
srat <- subset(srat, subset = is.na(Is_doublet))
srat

In [ ]:
srat[['Is_doublet']] <- !is.na(srat[['Is_doublet']])

In [ ]:
srat <- subset(srat, subset = Is_doublet != 'True')
srat

In [ ]:
srat@meta.data

In [ ]:
'nCount_RNA' %in% names(srat@meta.data)

In [ ]:
result <- GetMetadataFromSeurat(path)
result

In [ ]:
suffix <- tolower(get_suffix(path))
suffix

In [ ]:
seurat_object <- LoadSeurat(path)
seurat_object

In [ ]:
'umap' %in% names(seurat_object@reductions)

In [ ]:
genes <- rownames(seurat_object)
typeof(genes)
genes

In [ ]:
cells <- Cells(seurat_object)
typeof(cells)
cells

In [ ]:
pca <- Embeddings(object = seurat_object, reduction = "pca")
typeof(pca)
pca

In [ ]:
name <- load(path)
seurat_object <- UpdateSeuratObject(get(name))

In [ ]:
seurat_object

In [ ]:
features <- seurat_object[['RNA']]@var.features
typeof(features)
features

In [ ]:
#获取当前用的Idents
Idents(seurat_object)
levels(seurat_object)

In [ ]:
#总数目
nCells <- ncol(seurat_object)
nGenes <- nrow(seurat_object)

nCells
nGenes

In [ ]:
dim=1:10
class(dim)

In [ ]:
x <- 1:10
x

In [ ]:
class(x)

In [ ]:
devtools::install_github('JiekaiLab/dior')

In [4]:
source("../tools/benchmarks/clustering/seurat.R")
path = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad"


Attaching package: ‘dplyr’


The following object is masked from ‘package:Biobase’:

    combine


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following object is masked from ‘package:matrixStats’:

    count


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


here() starts at /ps/Machine-learning-development-environment-for-single-cell-sequencing-data-analyses



In [12]:
clustering <- function(path, labels, dims=1:10){ # labels: column name of labels in srat@meta.data
    srat <- tryCatch(
        LoadSeurat(path),
        error = function(e) {
            stop("The file format is not supported.")
            print(e)
        }
    )

    srat <- FindVariableFeatures(srat, selection.method = "vst")
    srat <- ScaleData(srat, features = rownames(srat))

    # PCA
    # srat <- RunPCA(srat, features = VariableFeatures(srat), ndims.print = 6:10, nfeatures.print = 10)
    srat <- RunPCA(srat, features = VariableFeatures(srat))

    srat <- FindNeighbors(srat, dims=dims)
    srat <- FindClusters(srat, resolution = 0.5)
    srat

    # TSNE
    srat <- RunTSNE(srat, dims=dims)
    # UMAP
    srat <- RunUMAP(srat, dims=dims)

    umap <- Embeddings(object = srat, reduction = "umap")

    list(labels=as.list(srat@meta.data[labels]), labels_pred=as.list(srat@meta.data["seurat_clusters"]), umap=umap)
}

In [13]:
result <- clustering(path, 'cluster.ids')
result

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
X -> counts; scale.data -> data

Finding variable features for layer counts

Centering and scaling data matrix

PC_ 1 
Positive:  Foxq1, Ly6d, Wfdc2, Fxyd3, Krt18, Aqp3, Gsta4, 2200002D01Rik, Krt7, Spint2 
	   Krt8, Krt19, 1110032A04Rik, Gsto1, Cldn4, Sprr1a, Ezr, Igfbp2, Gstm1, Lmo1 
	   Ivl, Mgst3, Klf5, Upk1b, Krt15, Tacstd2, Lgals3, Ptprf, Avpi1, Acaa1b 
Negative:  Lgals1, Vim, Sparc, Serpinh1, Col1a2, Serping1, Bgn, Dcn, Mgp, Plac9 
	   Selm, Col6a1, Htra1, Rcn3, Nbl1, Pcolce, Col6a2, Id3, Gadd45g, Mxra8 
	   Fstl1, Ppic, Nupr1, Cd302, Cst3, Cygb, Dkk3, Col5a2, Ecm1, Aebp1 
PC_ 2 
Positive:  S100a6, Serpinh1, Col1a2, Mgp, Dcn, Col6a1, Rcn3, Htra1, Serping1, Ppap2b 
	   Dkk3, Bgn, Col6a3, Pcolce, Ppic, Mxra8, Sparc, Fbln5, Col5a2, Ecm1 
	   Prelp, Nbl1, Col6a2, Fstl1, Thbs2, Aebp1, Spon1, Mmp23, Ctsk, Crispld2 
Negative:  Tyrobp, Cd52, H2-Eb1, H2-Aa, Ctss, Cd74, Ly86, Fcer1g, H2-DMa, Cd53 
	   Sfpi1, Srgn, Pld4, H2-

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 2392
Number of edges: 73531

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8789
Number of communities: 9
Elapsed time: 0 seconds


Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
03:29:18 UMAP embedding parameters a = 0.9922 b = 1.112

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by ‘spam’

03:29:18 Read 2392 rows and found 10 numeric columns

03:29:18 Using Annoy for neighbor search, n_neighbors = 30

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by ‘spam’

03:29:18 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

03:29:18 Writing NN index file to t

,umap_1,umap_2
10X_P4_3_AAAGTAGAGATGCCAG,8.967721,-2.2488236
10X_P4_3_AACTCCCGTCGGGTCT,6.261219,6.6866284
10X_P4_3_AACTCTTAGTTGCAGG,-8.660493,-0.1179474
10X_P4_3_AACTCTTTCATAACCG,4.522681,7.0582114
10X_P4_3_AAGACCTAGATCCGAG,5.769827,-14.9204139
10X_P4_3_AAGGAGCGTGCAACTT,9.130022,-3.2623500
10X_P4_3_AAGTCTGAGATAGTCA,-8.471845,1.5486604
10X_P4_3_ACAGCTATCATACGGT,-8.110715,-1.9064008
10X_P4_3_ACATACGCAGCTCCGA,-7.817172,-0.8551230
10X_P4_3_ACCAGTACACGTCTCT,8.298368,-3.0526160


In [14]:
result

,umap_1,umap_2
10X_P4_3_AAAGTAGAGATGCCAG,8.967721,-2.2488236
10X_P4_3_AACTCCCGTCGGGTCT,6.261219,6.6866284
10X_P4_3_AACTCTTAGTTGCAGG,-8.660493,-0.1179474
10X_P4_3_AACTCTTTCATAACCG,4.522681,7.0582114
10X_P4_3_AAGACCTAGATCCGAG,5.769827,-14.9204139
10X_P4_3_AAGGAGCGTGCAACTT,9.130022,-3.2623500
10X_P4_3_AAGTCTGAGATAGTCA,-8.471845,1.5486604
10X_P4_3_ACAGCTATCATACGGT,-8.110715,-1.9064008
10X_P4_3_ACATACGCAGCTCCGA,-7.817172,-0.8551230
10X_P4_3_ACCAGTACACGTCTCT,8.298368,-3.0526160
